In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv
import math
from sklearn.preprocessing import MinMaxScaler
import random

In [ ]:
data=read_csv("/content/drive/My Drive/sc_lab/heart2.xls")
data=pd.DataFrame(data)
scaler = MinMaxScaler()
scaler.fit(data)
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [ ]:
# d1=data[["age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal"]]
# d2=data["target"]
# d1=np.array(d1)
# d2=np.array(d2)
# data=np.array(data)
data=data.to_numpy()
random.shuffle(data)
print(data)


[[63.  1.  3. ...  0.  1.  1.]
 [37.  1.  2. ...  0.  2.  1.]
 [41.  0.  1. ...  0.  2.  1.]
 ...
 [51.  1.  2. ...  0.  2.  1.]
 [44.  1.  0. ...  0.  1.  0.]
 [53.  0.  0. ...  0.  2.  1.]]


In [ ]:
d2=data[:,-1]
d1=data[:,:-1]

In [ ]:
n=len(d1[0])
n

13

In [ ]:
def fun(input_data,target,weight_input,weight_hidden,bias_hidden,bias_output,L,no_hidden,no_output):

  for d in range(len(input_data)):
    net_input_hidden=[0.0 for p in range(no_hidden)]
    output_hidden=[0.0 for p in range(no_hidden)]

    for j in range(no_hidden):
      for i in range(n):
        net_input_hidden[j]+=input_data[d][i]*weight_input[j][i]
      net_input_hidden[j]+=bias_hidden[j]
      output_hidden[j]=1/(1+math.exp(-(net_input_hidden[j])))

    net_input_outputlayer=[0 for p in range(no_output)]
    output_outputlayer=[0 for p in range(no_output)]

    for k in range(no_output):
      for j in range(no_hidden):
        net_input_outputlayer[k]+=output_hidden[j]*weight_hidden[k][j]
      net_input_outputlayer[k]+=bias_output[k]
      output_outputlayer[k]=1/(1+math.exp(-(net_input_outputlayer[k])))

    error_output=[0 for p in range(no_output)]

    for j in range(no_output):
      error_output[j]=output_outputlayer[j]*(1-output_outputlayer[j])*((target[j]-output_outputlayer[j]))
    
    error_hidden=[0 for p in range(no_hidden)]

    for j in range(no_hidden):
      temp=0
      for k in range(no_output):
        temp+=weight_hidden[k][j]*error_output[k]

      error_hidden[j]=output_hidden[j]*(1-output_hidden[j])*temp

    for j in range(no_hidden):
      for k in range(no_output):
        weight_hidden[k][j]=weight_hidden[k][j]+L*error_output[k]*output_hidden[j]

    for j in range(n):
      for k in range(no_hidden):
        weight_input[k][j]=weight_input[k][j]+L*error_hidden[k]*input_data[d][j]

    
    for j in range(no_hidden):
      bias_hidden[j]=bias_hidden[j]+L*error_hidden[j]
    
    for j in range(no_output):
      bias_output[j]=bias_output[j]+L*error_output[j]

  return weight_input,weight_hidden,bias_hidden,bias_output



In [ ]:
def classification_report(cm):
    accuracy = 0
    total = np.sum(cm)
    diagonal = np.sum(np.diag(cm))
    accuracy=(diagonal/total)*100
    print("Accuracy : => ",(diagonal/total)*100)
    return accuracy

In [ ]:
no_hidden=5
no_output=1
block=int(len(data)/10)
print(block)
M=[[ 0 for i in range(2)] for i in range(2)]

for i in range(10):
  weight_input=[[1/(n*5) for i in range(n)] for j in range(no_hidden)]
  weight_hidden=[[1/5 for i in range(no_hidden)] for j in range(no_output)]
  bias_hidden=[1/6 for i in range(no_hidden)]
  bias_output=[1/6 for i in range(no_output)]
  L=0.2

  temp_x_data=[]
  temp_y_data=[]
  
  x_test=[]
  y_test=[]
  

  point=[]
  for j in range(block):
    point.append(block*i+j)

  
  for j in range(len(data)):
    if(j in point):
      x_test.append(d1[j])
      y_test.append(d2[j])
    else:
      temp_x_data.append(d1[j])
      temp_y_data.append(d2[j])


  
  matrix=[[0 for i in range(2)]for i in range(2)]
  
  for j in range(100):
    w1,w2,b1,b2=fun(temp_x_data,temp_y_data,weight_input,weight_hidden,bias_hidden,bias_output,L,no_hidden,no_output)
    weight_input,weight_hidden,bias_hidden,bias_output=w1,w2,b1,b2
  
  for x in range(len(x_test)):
    net_input_hidden=[0.0 for p in range(no_hidden)]
    output_hidden=[0.0 for p in range(no_hidden)]

    for k in range(no_hidden):
      for j in range(len(x_test[x])):
        net_input_hidden[k]+=x_test[x][j]*weight_input[k][j]
      net_input_hidden[k]+=bias_hidden[k]
      output_hidden[k]=1/(1+math.exp(-(net_input_hidden[k])))

    net_input_outputlayer=[0 for p in range(no_output)]
    output_outputlayer=[0 for p in range(no_output)]

    for k in range(no_output):
      for j in range(no_hidden):
        net_input_outputlayer[k]+=output_hidden[j]*weight_hidden[k][j]
      net_input_outputlayer[k]+=bias_output[k]
      output_outputlayer[k]=1/(1+math.exp(-(net_input_outputlayer[k])))
    c=0
    if(output_outputlayer[0]>0.5):
      c=1
    
    matrix[int(y_test[x])][c]+=1

  for p in range(2):
    for q in range(2):
      M[p][q]+=matrix[p][q]
  print("Fold ",i+1)
  print(matrix)
  accu=classification_report(matrix)

print("Final confusion matrix")
print(M)
print()
print("Average accuracy=>",classification_report(M))

30
Fold  1
[[0, 0], [0, 30]]
Accuracy : =>  100.0
Fold  2
[[0, 0], [0, 30]]
Accuracy : =>  100.0
Fold  3
[[0, 0], [0, 30]]
Accuracy : =>  100.0
Fold  4
[[0, 0], [0, 30]]
Accuracy : =>  100.0
Fold  5
[[0, 0], [0, 30]]
Accuracy : =>  100.0
Fold  6
[[0, 0], [0, 30]]
Accuracy : =>  100.0
Fold  7
[[0, 8], [0, 22]]
Accuracy : =>  73.33333333333333
Fold  8
[[0, 10], [0, 20]]
Accuracy : =>  66.66666666666666
Fold  9
[[0, 14], [0, 16]]
Accuracy : =>  53.333333333333336
Fold  10
[[0, 11], [0, 19]]
Accuracy : =>  63.33333333333333
Final confusion matrix
[[0, 43], [0, 257]]

Accuracy : =>  85.66666666666667
Average accuracy=> 85.66666666666667
